#  Singapore Transport Agent
**Author:** Parth Manekar  


##  Project Overview
This notebook implements an agentic workflow using **LangGraph** to answer user queries regarding Singapore's public transport. The agent is designed to be **context-aware**, factoring in real-time variables such as weather, traffic incidents, and peak-hour logic before generating a response.

##  Workflow Architecture
The solution uses a state-based graph architecture (`StateGraph`) to orchestrate the flow of data.
1.  **Intent Extraction:** A generic LLM (via Groq) analyzes the user's natural language query to classify intent (e.g., `bus_arrival`, `weather`, `traffic_area`) and extract entities (e.g., Bus Stop Codes).
2.  **Context Enrichment:** Before hitting the transport API, the state is enriched with environmental constraints:
    * **Time Context:** Determines if it is currently Peak/Off-Peak.
    * **Weather (Data.gov.sg):** Checks for rain to add travel advisories.
    * **Traffic (LTA):** Checks for congestion or accidents.
    * **Disruptions:** Checks MRT status.
3.  **Dynamic Routing:** Based on the intent, the agent routes to specific handlers (e.g., querying LTA DataMall for bus timings or filtering traffic incidents by location).
4.  **Response Generation:** A final node synthesizes the API data, environmental context, and user query into a natural language response.

##  Design Decisions & Assumptions
* **LangGraph over Chains:** Chosen for its cyclic and stateful nature, allowing for easier debugging and future expansion (e.g., adding a "correction" node if an API fails).
* **Deterministic Intent:** To reduce latency and token costs, I used the LLM *only* for intent extraction and final formatting. The core logic (Time/API fetching) is handled by deterministic Python functions.
* **Assumption:** The agent assumes the user is currently in Singapore (GMT+8) for time calculations.
* **Assumption:** "Next Bus" queries imply the immediate next arrival; however, the API returns up to 3 upcoming buses, which are processed to find the nearest one.

##  Deployment Strategy (Production)
To move this from a notebook to a production environment:
1.  **API Gateway (FastAPI):** Expose the `graph.invoke` method via a RESTful endpoint (e.g., `POST /chat`).
2.  **Caching (Redis):** LTA DataMall APIs have rate limits. I would implement a Redis cache (TTL: 1 min for Bus Arrival, 15 mins for Traffic) to serve frequent requests for the same bus stop without hitting the LTA origin server.
3.  **Async/Queues:** For high concurrency, API calls should be asynchronous (`aiohttp`). Long-running queries could be offloaded to a task queue (Celery/RabbitMQ).
4.  **Containerization:** Dockerize the application to ensure consistency across dev and prod environments.

In [61]:
# =========================
# SECTION 1: SETUP & STATE
# =========================

import os
import requests
from typing import TypedDict, Optional, Dict, Any
from datetime import datetime
import json

from langgraph.graph import StateGraph
from groq import Groq
import json




## Section 1: Setup & Configuration
Here we initialize the environment, setting up the **Groq** client (mimicking a standard chat completion interface) to handle Natural Language Processing tasks. We also define the `TransportState`, which acts as the central memory for our agent, storing the user query, extracted entities, and API results throughout the workflow.

In [ ]:
# =========================
# SECTION 1: SETUP & STATE 
# =======================

import os
import json
import time
from typing import TypedDict, Optional, Dict, Any
import requests


from groq import Groq

# ---------- GROQ SETUP ----------
# This wrapper mimics Gemini's .generate_content() so your existing code works.
class GroqModel:
    def __init__(self, api_key, model="openai/gpt-oss-safeguard-20b"):
        self.client = Groq(api_key=api_key)
        self.model = model

    def generate_content(self, prompt):
        try:
            # We force JSON mode if the prompt asks for JSON
            is_json = "json" in prompt.lower()
            
            chat_completion = self.client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                model=self.model,
                temperature=0, # Deterministic
                # Only enable strict JSON mode if keyword is present
                response_format={"type": "json_object"} if is_json else None
            )
            
            text_response = chat_completion.choices[0].message.content
            
            # Return an object that has a .text attribute (mimicking Gemini)
            return type('obj', (object,), {'text': text_response})
            
        except Exception as e:
            print(f"Groq API Error: {e}")
            # Return empty JSON string on failure to prevent crash
            return type('obj', (object,), {'text': "{}"})


GROQ_API_KEY = "<YOUR_API_KEY>"


gemini_model = GroqModel(api_key=GROQ_API_KEY, model="openai/gpt-oss-safeguard-20b")



In [ ]:
# ---------- LTA API KEY ----------
LTA_API_KEY = "<YOUR_LTA_API_KEY>"


In [ ]:

class TransportState(TypedDict):
    user_query: str
    intent: Optional[str]
    entities: Optional[Dict[str, Any]]

    # Context keys
    time_context: Optional[Dict]
    holiday_context: Optional[Dict]
    weather_context: Optional[Dict]
    disruption_context: Optional[Dict]
    

    bus_service_info: Optional[Dict] 

    api_response: Optional[Dict]
    final_answer: Optional[str]

##  Section 2: Context Enrichment Nodes
A smart agent shouldn't just look at data; it should understand the environment. In this section, we define nodes that fetch **constraints** and **environmental data**:
* **`time_context_node`**: Calculates if the current time falls under Singapore's "Peak Hour" windows.
* **`weather_context_node`**: Fetches real-time 2-hour forecasts from Data.gov.sg to warn users about rain.
* **`traffic_context_node`**: Analyzes LTA traffic incident data to gauge general road congestion.
* **`holiday_context_node`**: Checks against a list of known Public Holidays to adjust expectation of schedule frequency.

In [65]:
# =========================
# SECTION 2: CONTEXT NODES (UPDATED)
# =========================

def time_context_node(state: TransportState):
    """Calculates peak hour periods based on Singapore transport standards."""
    hour = datetime.now().hour
    minute = datetime.now().minute
    
    if (6 <= hour < 9) and not (hour == 6 and minute < 30):
         period = "morning_peak"
    elif 17 <= hour <= 20:
        period = "evening_peak"
    else:
        period = "off_peak"

    return {
        "time_context": {
            "hour": hour,
            "period": period,
            "is_peak": period != "off_peak"
        }
    }

SINGAPORE_HOLIDAYS = ["2025-01-01", "2025-01-29", "2025-01-30", "2025-03-31", "2025-05-01", "2025-05-12", "2025-06-07", "2025-08-09", "2025-10-20", "2025-12-25"]

def holiday_context_node(state: TransportState):
    today = datetime.now().strftime("%Y-%m-%d")
    return {"holiday_context": {"is_holiday": today in SINGAPORE_HOLIDAYS}}

def weather_context_node(state):
    """Fetches real 2-hour weather forecast from Data.gov.sg (NEA)."""
    try:
        
        url = "https://api.data.gov.sg/v1/environment/2-hour-weather-forecast"
        response = requests.get(url, timeout=5)
        data = response.json()

        if response.status_code != 200 or "items" not in data:
            return {"weather_context": {"condition": "Unknown", "impact": "N/A"}}

        forecasts = data.get("items", [])[0].get("forecasts", [])
        
        
        is_raining = any("Rain" in f["forecast"] or "Showers" in f["forecast"] for f in forecasts)
        condition = "Rain" if is_raining else "Cloudy/Clear"
        
        return {
            "weather_context": {
                "condition": condition,
                "impact": "Slippery roads and slower traffic expected." if is_raining else "Good travel conditions."
            }
        }
    except:
        
        return {"weather_context": {"condition": "Unknown", "impact": "N/A"}}


def disruption_context_node(state: TransportState):
    """Fetches real-time train alerts from DataMall2."""
    url = "https://datamall2.mytransport.sg/ltaodataservice/TrainServiceAlerts"
    headers = {"AccountKey": LTA_API_KEY, "accept": "application/json"}
    try:
        response = requests.get(url, headers=headers, timeout=5)
        data = response.json()
    except Exception:
        data = {"status": "unavailable"}
    return {"disruption_context": data}


def bus_service_info_node(state: TransportState):
    """
    Fetches General Bus Service Information with pagination support.
    """
    bus_no = str(state["entities"].get("bus_number"))
    if not bus_no or bus_no == "None":
        return {"bus_service_info": None}

    url = "https://datamall2.mytransport.sg/ltaodataservice/BusServices"
    headers = {"AccountKey": LTA_API_KEY, "accept": "application/json"}
    
    found_service = None
    skip = 0
    
    try:
        while True:
            # LTA uses $skip for pagination
            params = {"$skip": skip} 
            response = requests.get(url, headers=headers, params=params, timeout=10)
            
            if response.status_code != 200:
                break
                
            data = response.json()
            services = data.get("value", [])
            
            if not services:
                break # No more records
            
            # Search current batch
            target = next((s for s in services if s['ServiceNo'] == bus_no), None)
            if target:
                found_service = target
                break
                
            skip += 500 # Move to next batch
            
        return {"bus_service_info": found_service or "Not Found"}
        
    except Exception:
        return {"bus_service_info": "API Error"}
def traffic_context_node(state: TransportState):
    """Fetches traffic incidents to determine general road conditions."""
    url = "https://datamall2.mytransport.sg/ltaodataservice/TrafficIncidents"
    headers = {"AccountKey": LTA_API_KEY, "accept": "application/json"}
    
    try:
        response = requests.get(url, headers=headers, timeout=5)
        data = response.json()
        incidents = data.get("value", [])
        
        traffic_status = "Normal"
        if len(incidents) > 5:
            traffic_status = "High Congestion"
        elif len(incidents) > 2:
            traffic_status = "Moderate Traffic"
            
        return {
            "traffic_context": {
                "status": traffic_status,
                "incident_count": len(incidents),
                
                "major_incidents": [i["Message"] for i in incidents[:3]]
            }
        }
    except Exception:
        return {"traffic_context": {"status": "Unknown", "incident_count": 0}}
# =========================
def enrich_context(state: TransportState):
    state.update(time_context_node(state))
    state.update(holiday_context_node(state))
    state.update(weather_context_node(state))     
    state.update(disruption_context_node(state))
    state.update(traffic_context_node(state))
    state.update(bus_service_info_node(state))
    return state

In [66]:
def format_final_output(state):
    from datetime import datetime

    api_data = state.get("api_response", {})
    weather = state.get("weather_context", {})
    time_ctx = state.get("time_context", {})
    disruption = state.get("disruption_context", {})

    summary = "Transport information is currently unavailable."
    details = {}

    if "Services" in api_data and api_data["Services"]:
        svc = api_data["Services"][0]
        next_bus = svc.get("NextBus", {})

        eta_raw = next_bus.get("EstimatedArrival")
        eta_minutes = "N/A"
        eta_time = "N/A"

        if eta_raw:
            eta_dt = datetime.fromisoformat(eta_raw.replace("Z", "+00:00"))
            eta_time = eta_dt.strftime("%H:%M")
            eta_minutes = int(
                (eta_dt - datetime.now(eta_dt.tzinfo)).total_seconds() / 60
            )

        summary = f"Next bus {svc.get('ServiceNo')} is arriving in {eta_minutes} minutes."

        details = {
            "bus_stop": api_data.get("BusStopCode"),
            "service": svc.get("ServiceNo"),
            "arrival_time": eta_time,
            "crowding": next_bus.get("Load", "Unknown")
        }

    if weather.get("severity") in ["Moderate", "High"]:
        summary += " Minor delays may occur due to weather conditions."

    return {
        "summary": summary,
        "details": details,
        "context": {
            "time_of_day": time_ctx.get("period", "unknown"),
            "weather": weather.get("condition", "unknown"),
            "disruptions": (
                "Service disruptions detected"
                if disruption not in ({}, {"status": "unavailable"})
                else "No major disruptions"
            )
        },
        "data_source": state.get("data_source", "Live / Mock LTA Data")
    }


##  Section 3: Agent Intelligence & Routing
This section contains the core logic for the agent:
1.  **`extract_intent`**: Uses the LLM to convert unstructured text (e.g., "Is bus 12 coming to Pasir Ris?") into structured JSON (Intent: `bus_arrival`, Entity: `12`, Location: `Pasir Ris`).
2.  **`bus_arrival_node`**: The execution node. Based on the extracted intent, it dynamically queries the LTA DataMall API. It includes logic to handle specific constraints, such as filtering traffic incidents by specific locations (e.g., "Orchard Road").
3.  **`final_response_node`**: The synthesizer. It takes the raw JSON data from APIs and "humanizes" it, appending weather warnings if rain is detected.

In [ ]:
# =========================
# SECTION 3: AGENT LOGIC (UPDATED)
# =========================
def extract_intent(state: TransportState):
    """
    Improved intent + entity extraction for accurate routing
    """
    prompt = f"""
You are a Singapore transport assistant.

Analyze the user query:
"{state['user_query']}"

Classify intent STRICTLY into ONE of the following:
- bus_arrival → next bus / arrival time
- bus_info → route, operator, service details
- bus_frequency → peak or off-peak frequency
- traffic_area → traffic or jams in a specific area/road
- weather_only → rain, weather, umbrella
- train_disruption → MRT/LRT disruptions or line status
- general_help → vague help or travel planning
- fallback → unclear query

Extract entities if present:
- bus_stop_code (5 digits)
- bus_number
- location (e.g., Orchard Road, Jurong, Tampines)
- mrt_line (e.g., NEL, NSL)

Respond ONLY in valid JSON:
{{
  "intent": "...",
  "bus_stop_code": null,
  "bus_number": null,
  "location": null,
  "mrt_line": null
}}
"""
    response = gemini_model.generate_content(prompt)

    try:
        data = json.loads(
            response.text.strip()
            .replace("```json", "")
            .replace("```", "")
        )
    except Exception:
       
        data = {
            "intent": "fallback",
            "bus_stop_code": None,
            "bus_number": None,
            "location": None,
            "mrt_line": None
        }

   
    return {
        "intent": data.get("intent", "fallback"),
        "entities": data
    }



In [ ]:

def handle_area_traffic(state):
    """Filters traffic incidents based on the requested location."""
    location = (state.get("entities", {}).get("location") or "").lower()

    
    all_incidents = state.get("traffic_context", {}).get("major_incidents", [])
   
    relevant_incidents = []
    if location:
        relevant_incidents = [inc for inc in all_incidents if location in inc.lower()]
        status_msg = f"Traffic near {(location or 'your area').title()}"
    else:
        relevant_incidents = all_incidents
        status_msg = "General Traffic Update"

    return {
        "api_response": {
            "type": "traffic_area",
            "messages": relevant_incidents if relevant_incidents else ["No specific incidents reported in this area."],
            "status_title": status_msg
        }
    }

def handle_weather_only(state):
    """Pass-through: Weather is already in context."""
    return {"api_response": {"type": "weather_only"}}

def handle_train_disruption(state):
    """Pass-through: Disruption is already in context."""
    return {"api_response": {"type": "train_disruption"}}


# =========================
# UPDATED NODE: BUS_ARRIVAL_NODE
# =========================

def bus_arrival_node(state: TransportState):
    intent = state.get("intent")
    entities = state.get("entities", {})
    
    # --- ROUTING TO HANDLERS ---
    if intent == "traffic_area":
        return handle_area_traffic(state)

    elif intent == "weather_only":
        return handle_weather_only(state)

    elif intent == "train_disruption":
        return handle_train_disruption(state)

    # --- BUS ARRIVAL LOGIC ---
    stop_code = entities.get("bus_stop_code")
    bus_no = entities.get("bus_number")

    # If the user wants generic info (route/frequency) or general help, skip the arrival API
    if intent in ["bus_info", "bus_frequency", "general_help"]:
        return {"api_response": {"info": "Skipped arrival query for non-arrival intent."}}
        
    # Valid arrival query requires a stop code
    if not stop_code:
        return {"api_response": {"error": "No bus stop code provided."}}

    url = "https://datamall2.mytransport.sg/ltaodataservice/v3/BusArrival"
    headers = {"AccountKey": LTA_API_KEY, "accept": "application/json"}
    
    params = {"BusStopCode": stop_code}
    if bus_no:
        params["ServiceNo"] = bus_no

    try:
        response = requests.get(url, headers=headers, params=params, timeout=10)
        data = response.json() if response.status_code == 200 else {"error": f"Status {response.status_code}"}
    except Exception as e:
        data = {"error": str(e)}

    return {"api_response": data}

In [69]:
def final_response_node(state: TransportState):
    intent = state.get("intent")
    service = state.get("bus_service_info", {})
    api_response = state.get("api_response", {})

    weather = state.get("weather_context", {})
    traffic = state.get("traffic_context", {})
    disruption = state.get("disruption_context", {})
    time_ctx = state.get("time_context", {})

    from datetime import datetime

    # -------------------------
    # Universal Weather Alert
    # -------------------------
    weather_alert = ""
    if weather.get("condition") == "Rain" or weather.get("severity") in ["Moderate", "High"]:
        weather_alert = (
            "\n\n---\n"
            "### ☔ Weather Update\n"
            f"**Condition:** {weather.get('condition', 'Unknown')}\n"
            f"**Advisory:** {weather.get('impact', 'Please carry an umbrella.')}"
        )

    # =====================================================
    # 1. BUS ARRIVAL
    # =====================================================
    if intent == "bus_arrival":
        response_text = "I checked the bus timings for you, but the arrival details are currently unavailable."

        if api_response.get("Services"):
            svc = api_response["Services"][0]
            next_bus = svc.get("NextBus", {})
            est = next_bus.get("EstimatedArrival")

            mins, arrival_time = "N/A", "Unknown"
            if est:
                dt = datetime.fromisoformat(est)
                now = datetime.now(dt.tzinfo)
                mins = max(0, int((dt - now).total_seconds() / 60))
                arrival_time = dt.strftime("%I:%M %p")

            response_text = (
                "Your bus is on the way — here are the latest details:\n\n"
                f"## 🚌 Bus {svc.get('ServiceNo')} – Arrival at Stop {api_response.get('BusStopCode')}\n\n"
                f"**Next Bus:** Arriving in **{mins} minutes** ({arrival_time})\n"
                f"**Crowding:** {next_bus.get('Load', 'Unknown')}"
            )
        response_text += weather_alert

    # =====================================================
    # 2. BUS SERVICE INFO / FREQUENCY (Merged)
    # =====================================================
    elif intent in ["bus_info", "bus_frequency", "bus_service_info"]:
        if isinstance(service, dict) and service.get("ServiceNo"):
            response_text = (
                f"Here’s the info for Bus Service {service.get('ServiceNo')}:\n\n"
                f"## 🚌 Service {service.get('ServiceNo')} Details\n\n"
                f"**Route:** {service.get('OriginCode')} ↔ {service.get('DestinationCode')}\n"
                f"**Operator:** {service.get('Operator', 'Unknown')}\n\n"
                f"### ⏱️ Frequency\n"
                f"**AM Peak:** {service.get('AM_Peak_Freq', '-')} mins\n"
                f"**Off-Peak:** {service.get('AM_Offpeak_Freq', '-')} mins\n"
                f"**PM Peak:** {service.get('PM_Peak_Freq', '-')} mins\n"
            )
        else:
            response_text = f"I couldn't find specific details for that bus service. (Received: {state['entities'].get('bus_number', 'None')})"

    # =====================================================
    # 3. SPECIFIC AREA TRAFFIC
    # =====================================================
    elif intent == "traffic_area":
        location = state["entities"].get("location", "the area")
        msgs = api_response.get("messages", [])
        
        if msgs:
            bullet_points = "\n".join([f"- {msg}" for msg in msgs])
            details = f"**Incidents:**\n{bullet_points}"
        else:
            details = "No major incidents reported here right now."

        response_text = (
           f"## 🚦 Traffic Check: {(location or 'your area').title()}\n\n"
            f"{details}\n\n"
            f"**General Context:** {traffic.get('status', 'Normal Traffic')}"
        )
        response_text += weather_alert

    # =====================================================
    # 4. WEATHER ONLY
    # =====================================================
    elif intent in ["weather_only", "weather"]:
        is_rain = weather.get("condition") == "Rain"
        response_text = (
            "## 🌦️ Weather Update\n\n"
            f"**Current Condition:** {weather.get('condition', 'Unknown')}\n"
            f"**Advisory:** {weather.get('impact', 'No data')}\n\n"
            f"{'⚠️ Umbrella recommended!' if is_rain else '✅ No umbrella needed.'}"
        )

    # =====================================================
    # 5. TRAIN / GENERAL DISRUPTION
    # =====================================================
    elif intent in ["train_disruption", "travel_conditions"]:
        mrt_status = "Normal Service"
        if disruption.get("Status") == 2:
            mrt_status = f"⚠️ {disruption.get('Message', 'Disrupted')}"
        elif disruption.get("value"): # Sometimes LTA returns a list in 'value'
             mrt_status = "⚠️ Alerts detected (Check LTA for details)"

        response_text = (
            "## 🚆 Transport Network Status\n\n"
            f"**MRT/LRT:** {mrt_status}\n"
            f"**Traffic Overall:** {traffic.get('status', 'Normal')}\n"
            f"**Weather:** {weather.get('condition', 'Unknown')}"
        )

    # =====================================================
    # 6. GENERAL HELP
    # =====================================================
    elif intent in ["general_help", "general_travel_chat"]:
        response_text = (
            "👋 Hi there! I'm your Singapore Transport Assistant.\n\n"
            "I can help you with:\n"
            "- **Bus Arrivals:** \"When is bus 176 arriving at 20251?\"\n"
            "- **Bus Info:** \"Frequency of bus 107M?\"\n"
            "- **Traffic:** \"Any jams at Orchard?\"\n"
            "- **Weather:** \"Is it raining?\""
        )

    # =====================================================
    # FALLBACK
    # =====================================================
    else:
        response_text = (
            "Sorry, I didn't quite catch that specific request. "
            "Could you try asking about a specific bus, stop, or location?"
        )

    return {"final_answer": response_text}

##  Section 4: Graph Assembly
We use **LangGraph** to wire the components together. The flow is linear but modular:
`Start` -> `Intent Extraction` -> `Context Enrichment` -> `API Fetching` -> `Final Answer` -> `End`

This structure allows for "Separation of Concerns"—the node fetching the weather doesn't need to know about the node fetching the bus times.

In [74]:
# =========================
# SECTION 4: GRAPH ASSEMBLY
# =========================

graph = StateGraph(TransportState)

graph.add_node("intent", extract_intent)
graph.add_node("context", enrich_context)
graph.add_node("bus_api", bus_arrival_node)
graph.add_node("answer", final_response_node)

graph.set_entry_point("intent")
graph.add_edge("intent", "context")
graph.add_edge("context", "bus_api")
graph.add_edge("bus_api", "answer")

app = graph.compile()


In [71]:
import re

def format_for_cli(text: str) -> str:
    # Remove markdown bold
    text = re.sub(r"\*\*(.*?)\*\*", r"\1", text)

    # Normalize spacing
    lines = [line.strip() for line in text.splitlines() if line.strip()]

    return "\n".join(lines)


def print_clean_output(result):
    print("\n" + "=" * 50)
    output = result.get("final_answer", "No output")
    print(format_for_cli(output))
    print("=" * 50 + "\n")


# ---------- SINGLE TEST ----------
result = app.invoke({
    "user_query": "When is the next bus 176 arriving at stop 20251?"
})

print_clean_output(result)



Your bus is on the way — here are the latest details:
## 🚌 Bus 176 – Arrival at Stop 20251
Next Bus: Arriving in 0 minutes (04:17 PM)
Crowding: SEA
---
### ☔ Weather Update
Condition: Rain
Advisory: Slippery roads and slower traffic expected.



## Section 5: Multi-User Simulation
Per the assessment requirements, we simulate **10 distinct user profiles** querying the agent.
* **Scope:** The queries range from specific bus arrival times to general weather checks and traffic inquiries.
* **Rate Limiting:** A `time.sleep` mechanism is implemented between requests to respect the LTA DataMall API rate limits during this batch process.

In [72]:
# =========================
# SECTION 5: PART 2 - SIMULATION LOOP (With Rate Limit Fix)
# =========================
import time  # <--- IMPORT THIS

simulation_queries = [
    "When is the next bus 176 arriving at stop 20251?",
    "Is there any heavy traffic or jams right now?",
    "Tell me about bus service 15.",
    "Is it raining? Should I bring an umbrella?",
    "When is the next bus arriving at stop 83139?",
    "Are there any train disruptions on the NEL line?",
    "What is the frequency of bus 107M during peak hours?",
    "When is the next bus 177 arriving at stop 20251?",
    "Check bus arrival for stop 00000",
    "Hi, I need help with my travel plans."
]

print(f"🚀 Starting Simulation of {len(simulation_queries)} User Requests...\n")

for i, query in enumerate(simulation_queries, 1):
    print(f"👤 USER {i}: \"{query}\"")
    print("-" * 60)
    
    try:
        # Invoke the agent
        result = app.invoke({"user_query": query})
        
        # Print the Agent's Response
        print(f"🤖 AGENT:\n{result.get('final_answer', 'No response generated.')}")
        
    except Exception as e:
        print(f"❌ ERROR: {e}")
        
    print("\n" + "="*60 + "\n")
    
    
    # The error asked to wait ~22 seconds. We wait 30s to be safe.
    if i < len(simulation_queries):
        print("⏳ Waiting 10s to respect  API rate limits...")
        time.sleep(10)

🚀 Starting Simulation of 10 User Requests...

👤 USER 1: "When is the next bus 176 arriving at stop 20251?"
------------------------------------------------------------
🤖 AGENT:
Your bus is on the way — here are the latest details:

## 🚌 Bus 176 – Arrival at Stop 20251

**Next Bus:** Arriving in **0 minutes** (04:17 PM)
**Crowding:** SEA

---
### ☔ Weather Update
**Condition:** Rain
**Advisory:** Slippery roads and slower traffic expected.


⏳ Waiting 10s to respect  API rate limits...
👤 USER 2: "Is there any heavy traffic or jams right now?"
------------------------------------------------------------
🤖 AGENT:
## 🚦 Traffic Check: Your Area

**Incidents:**
- No specific incidents reported in this area.

**General Context:** Normal Traffic

---
### ☔ Weather Update
**Condition:** Rain
**Advisory:** Slippery roads and slower traffic expected.


⏳ Waiting 10s to respect  API rate limits...
👤 USER 3: "Tell me about bus service 15."
-----------------------------------------------------------